In [ ]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv(dotenv_path="/home/mahdi/Desktop/Langchain/.env")

# Set up OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key

# Set up LangSmith (optional)
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
if langsmith_api_key:
    os.environ["LANGSMITH_API_KEY"] = langsmith_api_key

langsmith_tracing = os.getenv("LANGSMITH_TRACING")
if langsmith_tracing:
    os.environ["LANGCHAIN_TRACING_V2"] = langsmith_tracing

# Initialize OpenAI LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

print("OpenAI API key:", "Set" if os.getenv("OPENAI_API_KEY") else "Not set")
print("LangSmith key:", "Set" if os.getenv("LANGSMITH_API_KEY") else "Not set")
print("LangChain tracing V2:", os.getenv("LANGCHAIN_TRACING_V2"))

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
#built in tools
wiki_wrapper = WikipediaAPIWrapper(top_k_results = 1 , doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [ ]:
#custom tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS 
from langchain_community.embeddings import OllamaEmbeddings 
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

vectoredb = FAISS.from_documents(documents = docs, embedding = OllamaEmbeddings(model = "llama2"))
retriever = vectoredb.as_retriever()
retriever 

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever=retriever, name="langchain_docs", description="Retrieves documents from LangChain documentation")

In [ ]:
retriever_tool.name

In [ ]:
#built in tools
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [ ]:
#combine tools

tools = [retriever_tool, arxiv_tool, wiki_tool]

In [ ]:
tools

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)
llm

In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
'''init Agent to use tools , which will give the answer depending on the tools order '''
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

In [ ]:
# Agent Execution

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor



In [ ]:
agent_executor.invoke({"input": "What is LangSmith?"})